# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split

2024-11-30 03:38:09.066672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# your code here
df = pd.read_csv('tic-tac-toe.csv' )

# Inspect the dataset
df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [11]:
df.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

In [13]:
df.shape

(958, 10)

In [15]:
# Convert the categorical values to numeric
label_encoders = {}
for column in df.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le 

df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1


In [17]:
# Separate the inputs and output
X = df.iloc[:, :-1]  # All columns except the last one
y = df.iloc[:, -1]   # The last column

In [19]:
# Normalize the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
X_normalized[:5]

array([[1. , 1. , 1. , 1. , 0.5, 0.5, 1. , 0.5, 0.5],
       [1. , 1. , 1. , 1. , 0.5, 0.5, 0.5, 1. , 0.5],
       [1. , 1. , 1. , 1. , 0.5, 0.5, 0.5, 0.5, 1. ],
       [1. , 1. , 1. , 1. , 0.5, 0.5, 0.5, 0. , 0. ],
       [1. , 1. , 1. , 1. , 0.5, 0.5, 0. , 0.5, 0. ]])

In [21]:
# Convert back to a df if needed
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns)

print("\nShape of inputs (X) and output (y):")
print("Inputs:", X_normalized_df.shape)
print("Output:", y.shape)


Shape of inputs (X) and output (y):
Inputs: (958, 9)
Output: (958,)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [39]:
# your code here

# Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Create a Sequential model
model = tf.keras.Sequential()

# Add layers to the model
# Input layer
model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))

# Hidden layers with ReLU activation
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))

# Output layer with Softmax activation
model.add(tf.keras.layers.Dense(len(y.unique()), activation='softmax'))  # Assumes y has unique classes

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fit the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model with the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save the model
model.save("tic-tac-toe.keras")
print("\nModel saved as tic-tac-toe.keras.")


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6271 - loss: 0.6661 - val_accuracy: 0.6818 - val_loss: 0.6315
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6463 - loss: 0.6336 - val_accuracy: 0.6818 - val_loss: 0.6312
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6596 - loss: 0.6201 - val_accuracy: 0.6753 - val_loss: 0.6310
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6197 - loss: 0.6372 - val_accuracy: 0.6688 - val_loss: 0.6308
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.6496 - loss: 0.6273 - val_accuracy: 0.6623 - val_loss: 0.6299
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6391 - loss: 0.6257 - val_accuracy: 0.6558 - val_loss: 0.6305
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6784 - loss: 0.6057 - val_accuracy: 0.6494 - val_loss: 0.6298
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6790 - loss: 0.6061 - val_accuracy: 0.6429 - val_l

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [43]:
# your code here

# Load the saved model
loaded_model = tf.keras.models.load_model("tic-tac-toe.keras")
print("Model loaded successfully.")

# Select a few random rows from the test dataset
num_samples = 5  # Number of random samples to check
random_indices = np.random.choice(X_test.shape[0], num_samples, replace=False)
random_X_samples = X_test[random_indices]
random_y_samples = y_test.iloc[random_indices]

# Make predictions
predictions = loaded_model.predict(random_X_samples)
predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

# Compare predictions with actual labels
for i, idx in enumerate(random_indices):
    print(f"Sample {i+1}:")
    print(f"Input Features: {X_test[idx]}")
    print(f"Actual Label: {random_y_samples.iloc[i]}")
    print(f"Predicted Label: {predicted_classes[i]}")
    print(f"Confidence: {predictions[i][predicted_classes[i]]:.4f}")
    print("-" * 30)

Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Sample 1:
Input Features: [0.5 1.  1.  0.  0.5 1.  0.  0.5 1. ]
Actual Label: 1
Predicted Label: 1
Confidence: 0.8788
------------------------------
Sample 2:
Input Features: [0.5 1.  0.  0.  1.  0.5 0.  1.  0. ]
Actual Label: 1
Predicted Label: 1
Confidence: 0.8005
------------------------------
Sample 3:
Input Features: [0.  0.5 1.  1.  0.5 1.  1.  0.5 0.5]
Actual Label: 0
Predicted Label: 0
Confidence: 0.6356
------------------------------
Sample 4:
Input Features: [1.  0.5 1.  1.  0.5 0.5 0.5 1.  1. ]
Actual Label: 0
Predicted Label: 0
Confidence: 0.5813
------------------------------
Sample 5:
Input Features: [0.5 0.5 0.5 0.  1.  0.  0.  1.  1. ]
Actual Label: 0
Predicted Label: 1
Confidence: 0.8976
------------------------------


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [47]:
# your code here

# Split the training and test data again if necessary
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Create a Sequential model with more layers
improved_model = tf.keras.Sequential()

# Input layer
improved_model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))

# Add more hidden layers with ReLU activation
improved_model.add(tf.keras.layers.Dense(32, activation='relu'))  # Increased neurons
improved_model.add(tf.keras.layers.Dense(32, activation='relu'))
improved_model.add(tf.keras.layers.Dense(16, activation='relu'))

# Output layer with Softmax activation
improved_model.add(tf.keras.layers.Dense(len(y.unique()), activation='softmax'))

# Adjust learning rate and compile the model
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Custom learning rate
improved_model.compile(optimizer=custom_optimizer,
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Fit the model with adjusted number of epochs
history = improved_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the improved model
test_loss, test_accuracy = improved_model.evaluate(X_test, y_test, verbose=0)
print(f"\nImproved Test Loss: {test_loss:.4f}")
print(f"Improved Test Accuracy: {test_accuracy:.4f}")

# Save the improved model
improved_model.save("tic-tac-toe-improved.keras")
print("\nImproved model saved as tic-tac-toe-improved.keras.")


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.6027 - loss: 0.6742 - val_accuracy: 0.6818 - val_loss: 0.6222
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6326 - loss: 0.6396 - val_accuracy: 0.6818 - val_loss: 0.6210
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6315 - loss: 0.6316 - val_accuracy: 0.6818 - val_loss: 0.6145
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6377 - loss: 0.6172 - val_accuracy: 0.6883 - val_loss: 0.6127
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6648 - loss: 0.5894 - val_accuracy: 0.7078 - val_loss: 0.6065
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6890 - loss: 0.5886 - val_accuracy: 0.7143 - val_loss: 0.6026
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6877 - loss: 0.5743 - val_accuracy: 0.7143 - val_loss: 0.5993
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6940 - loss: 0.5647 - val_accuracy: 0.6948 - 

**Which approach(es) did you find helpful to improve your model performance?**

# your answer here
- Increasing the number of layers and neurons enhanced the model's capacity to learn complex patterns in the dataset.
- Using a custom learning rate of 0.001 (a common starting point) helped the Adam optimizer converge efficiently without overshooting or oscillating.
- Training for 100 epochs allowed the model to fully utilize the training data and reach its performance ceiling.
- Tracking validation loss and accuracy ensured that the model didn't overfit while increasing epochs.